## Skript zum Erzeugen der benötigten Stopwords sowie der N-Grams

In [ ]:
from bs4 import BeautifulSoup
from utils import preprocessing
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from collections import Counter
from stop_words import get_stop_words
from nltk.util import ngrams
import json

### Einlesen der Cases

Die Datensatz wird in ein Pandas Dataframe eingelesen

In [ ]:
cases_df = pd.read_json('../2019-02-19_oldp_cases.json', lines=True)

Der Pandas Dataframe wird in eine Python Liste umgeformt und anschließend gelöscht, um Arbeitsspeicher freizugeben

In [ ]:
cases = []
for i in range(len(cases_df)) : 
    cases.append(cases_df.loc[i].to_dict())
    
del cases_df

### Cleanen der eingelesenen Cases 

Methode: clean

IN: Der zu cleanende Content

OUT: Der gecleante Content

In dieser Funktion werden unter anderem die HTML-Tags sowie Whitespaces entfernt. Zudem werden zwischen Aufzählungen und dem nachfolgenden Text jeweils ein Punkt sowie ein Leerzeichen eingefügt, um die Aufzählung vom Text zu separieren. Ist bereits ein Punkt nach der Aufzählung vorhanden, so wird nur ein Leerzeichen ergänzt.
Potentielle Tippfehler, wie etwa ",." werden durch "." ersetzt, ":." durch ":"

In [ ]:
def clean(content):
    soup = BeautifulSoup(content)
    content = soup.get_text()
    content = preprocessing.remove_whitespace(content)
    
    content = re.sub(r'(\d)([a-zA-ZäöüÄÖÜß])', "\g<1>. \g<2>", content)
    content = re.sub(r'(\d)(\.)([a-zA-ZäöüÄÖÜß])', "\g<1>\g<2> \g<3>", content)
    content = re.sub(r'(\d)(\„)([a-zA-ZäöüÄÖÜß])', "\g<1>. \g<2>\g<3>", content)
    
    content = content.replace(",.",".")
    content = content.replace(":.",":")
      
    return content

Ausführen der clean-Methode auf den Content jedes einzelnen Cases und anhängen der gecleanten Inhalte an den String "text".
Dieser String wird im Anschluss für die Generierung der Stopwords und N-Grams benötigt.

In [ ]:
text = ''
i = 0
for case in cases:
    text += clean(case['content'])
    i = i+1
    
    if(i % 10000 == 0):
        print(i)

Ersetzen der "\n" durch Leerzeichen in "text" und anschließend Großschreibung zu Kleinschreibung abändern.

In [ ]:
text = text.replace("\n"," ")
text = text.lower()

### Erzeugen der Stopwords

Aufsplitten von "text" in einzelne Wörter mit Hilfe des Word-Tokenizers von NLTK

In [ ]:
words = nltk.word_tokenize(text)

Löschen der Variablen "text" und "content", da diese im Weiteren nicht mehr benötigt werden. Dadurch wird eine Freigabe des Arbeitsspeichers ermöglicht

In [ ]:
del text
del cases

Zählen der Auftrittshäufigkeit der einzelnen Wörter

In [ ]:
c = Counter(words)
n_words = 1000

Ausgeben der häufigsten 1000 Wörter 

In [ ]:
most_common = c.most_common(n_words)

In [ ]:
most_common

Nach Überprüfung der Ausgabe stellt sich heraus, dass bis Index 46 ('kläger', 1000777) ausschließlich Stopwords auftreten

In [ ]:
index = most_common.index(('kläger', 1000777))
print(index)

Stopwords aus "stop_words" in "predefined_stop_words" ablegen

In [ ]:
predefined_stop_words = set(get_stop_words('german'))
#print(predefined_stop_words)

Stopwords aus "nltk.corpus" in "nltk_stop_words" ablegen

In [ ]:
nltk_stop_words = set(stopwords.words('german'))
#print(nltk_stop_words)

Wörter aus "most_common" bis zum ermittelten Index in "most_common_only_word" ablegen.
Anschließend alle Wörter des Korpuses, welche weniger als 3 Zeichen haben ebenfalls als Stopwords deklarieren und ebenfalls in "most_common_only_word" ablegen.
Durch Verwendung von ".union" ist sichergestellt, dass alle Stopwords nur einmal vorkommen.

In [ ]:
most_common_only_word = set([word[0] for word in most_common[:index]])

most_common_only_word = most_common_only_word.union([word for word in words if len(word)<3])

print(len(most_common_only_word))

#print(most_common_only_word)

Möglichkeit um zusätzliche benutzerdefinierte Stopwords hinzuzufügen.

In [ ]:
#zusätzliche Stopwords hinzufügen
additional_custom_stopwords = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ',,', '[', ']', 'vgl', '...', \
                               'ovg', 'nrw','de',"a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "bzw", \
                               "k", "l", "m", "n", "o", "p", "q", "r","s", "t", "u", "v", "w", "x", "y", "z"]


most_common_only_word = most_common_only_word.union(additional_custom_stopwords)

print(len(most_common_only_word))
#print(most_common_only_word)

Alle vorher definierten Variablen, welche Stopwords enthalten, in "stop_words" mithilfe von ".union" ablegen und "stop_words" anschließend in eine Liste umformen.

In [ ]:
stop_words = nltk_stop_words.union(predefined_stop_words.union(most_common_only_word))
stop_words = list(stop_words)

"stop_words" in ein JSON-File ablegen

In [ ]:
with open('../all_stop_words.json', 'w', encoding='utf-8') as f:
    json.dump(stop_words, f, ensure_ascii=False)

Aus der Wort-Liste alle Stopwords herausfiltern und in "filtered_words" ablegen

In [ ]:
filtered_words = [word for word in words if word not in stop_words]   
#print(filtered_words)

"filtered_words" in ein JSON-File ablegen

In [ ]:
with open('../all_filtered_words.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_words, f, ensure_ascii=False)

Variable "words" freigeben

In [ ]:
del words

### Zum Einlesen der "filtered_words" falls Kernel stirbt

In [ ]:
with open('../all_filtered_words.json', encoding='utf-8') as f:
    filtered_words = json.load(f)

### -------------------------------------------

### Erzeugen der N-Grams 

Methode: getNgrams

IN: Liste der Wörter "words", n gibt an welches N-Gram erzeugt werden soll (3-Gram, 5-Gram etc.)

OUT: Ermittelte N-Grams als Pandas Series

In [ ]:
def getNgrams(words ,n):
    
    Ngrams = ngrams(words, n)
    
    return(pd.Series(Ngrams))

Methode: edit_to_nice_df

IN: Pandas Series "pdseries", welche durch Methode "getNgrams" zurückgegeben wurde

OUT: Pandas Dataframe mit den entsprechenden N-Grams und deren Häufigkeit

Hier wird die Methode "value_counts()" der übergebene Pandas Series aufgerufen und die so erhaltene Häufigkeit der N-Grams in der Spalte "Counts" eines Pandas Dataframe abgelegt, zusammen mit dem entsprechenden N-Grams in der Spalte "N_Grams".
Anschließend wird der Dataframe neu indiziert und zurückgegeben.

In [ ]:
def edit_to_nice_df(pdseries):
    
    df = pd.DataFrame(pdseries.value_counts(),columns=['Counts'])
    df['N_Grams'] = df.index
    df.reset_index(drop=True, inplace=True)
    
    return df

Methode: export_Ngrams_JSON

IN: Der Pfad in dem das JSON-File abgespeichert werden soll "path", der Name des JSON-Files "filename" sowie das N-Gram selbst "ngram"

Hier wird zunächst der Pfad, der Dateiname sowie die Datei-Endung zusammengesetzt. Anschließend wird mithilfe der von Pandas Dataframe zur Verfügung gestellten "to_json" Methode das N-Gram in das entsprechende JSON-File geschrieben.

In [ ]:
def export_Ngrams_JSON(path, filename, ngram):
        full_filepath = path+filename+".json"
        ngram.to_json(full_filepath, force_ascii=False, default_handler=str, orient='table', index = True)

Erzeugung der 3- und 5-Grams.

Bei 3-Grams werden hierfür alle erzeugten 3-Grams in zwei JSON-Dateien abgelegt. Diese Aufteilung erleichtert das spätere Einlesen und Pushen an Solr.
Bei 5-Grams werden aufgrund der erheblichen Dateigröße nur die häufigsten 75% der 5-Grams in zwei JSON-Dateien geschrieben. Die Aufteilung erleichtert auch hier das Einlesen und Pushen an Solr

In [ ]:
export_path_grams = "../"
for i in [3, 5]:
    if i==3:
        drigrams_df = edit_to_nice_df(getNgrams(filtered_words, i))
        export_Ngrams_JSON(export_path_grams, str(i)+"grams_df_1", drigrams_df[:int(len(drigrams_df.index) * .5)])
        export_Ngrams_JSON(export_path_grams, str(i)+"grams_df_2", drigrams_df[int(len(drigrams_df.index) * .5):])
        del drigrams_df
    else:
        fivegrams_df = edit_to_nice_df(getNgrams(filtered_words, i))
        fivegrams_df = fivegrams_df[:int(len(fivegrams_df.index) * .75)]
        export_Ngrams_JSON(export_path_grams, str(i)+"grams_df_1", fivegrams_df[:int(len(fivegrams_df.index) * .5)])
        export_Ngrams_JSON(export_path_grams, str(i)+"grams_df_2", fivegrams_df[int(len(fivegrams_df.index) * .5):])
        del fivegrams_df